In [1]:
import glob
import nibabel as nib
from nistats.first_level_model import FirstLevelModel
import numpy as np
import os
import pandas as pd
import pickle
import re
from ggplot import *

/home/groups/russpold/software/miniconda/envs/fmri/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/groups/russpold/software/miniconda/envs/fmri/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/groups/russpold/software/miniconda/envs/fmri/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/groups/russpold/software/miniconda/envs/fmri/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/groups/russpold/software/miniconda/envs/fmri/lib/python3.5/importlib/_bootstrap.py:2

In [2]:
data_loc = os.environ['DATA_LOC']
server_scripts = os.environ['SERVER_SCRIPTS']
pe = True

## Check correlation between high and low var PE regressors for all subjects in all runs 
becase the high PE looked strange while low PE looked reasonable

In [3]:
design_matrices = glob.glob('%s/derivatives/nistats/level_1/sub-*/sub-*_run-*_level1_pe_design_matrix.csv'%(data_loc))
out_df = pd.DataFrame(columns=['subnum', 'runnum', 'cor_hpe_lpe'])
for dm in design_matrices:
    subnum = re.findall('\d+', os.path.basename(dm))[0]
    runnum = re.findall('\d+', os.path.basename(dm))[1]
    design_matrix = pd.read_csv(dm)
    try:
        out_df = out_df.append({'subnum': subnum, 'runnum': runnum, 'cor_hpe_lpe': design_matrix['hpe'].corr(design_matrix['lpe'])}, ignore_index=True)
    except:
        continue

Not really correlated for anyone

In [4]:
out_df.sort_values(by=['cor_hpe_lpe'])

,subnum,runnum,cor_hpe_lpe
19,100250,005,-0.318641
346,311047,004,-0.275160
204,200148,006,-0.195042
131,405027,005,-0.164756
119,409874,006,-0.159944
102,100247,006,-0.155606
289,200211,004,-0.152061
133,311283,001,-0.149222
246,100143,001,-0.144689
314,411477,006,-0.137232


## Other correlated regressors in the design matrix

More generally check correlations between task-related regressors in the design matrix. Extract the highest correlation for each regressor and filter by absolute values higher than 0.5

In [5]:
cols = ['hpe', 'lpe', 'm1','m1_rt', 'm2', 'm2_rt', 'm3', 'm3_rt', 'm4', 'm4_rt']
out_df = pd.DataFrame(columns=['subnum','runnum','col1', 'col2', 'cor'])

for dm in design_matrices:
    subnum = re.findall('\d+', os.path.basename(dm))[0]
    runnum = re.findall('\d+', os.path.basename(dm))[1]
    design_matrix = pd.read_csv(dm)
    try:
        cor_m = design_matrix[['hpe', 'lpe', 'm1','m1_rt', 'm2', 'm2_rt', 'm3', 'm3_rt', 'm4', 'm4_rt']].corr()
        for col in cols:
            cur_col = abs(cor_m[col]).drop(labels=col)
            out_df = out_df.append({'subnum': subnum, 'runnum': runnum,'col1': col, 'col2': cur_col[cur_col == max(cur_col)].index[0], 'cor': max(cur_col)}, ignore_index=True)
    except:
        continue

Most of the high correlations are between specific machines and their RTs. Except for the three strange runs with perfectly correlated PEs.

In [18]:
out_df.query('abs(cor)>0.5').sort_values(by=['cor'], ascending=False).drop_duplicates(subset='cor', keep='first')

,subnum,runnum,col1,col2,cor
2712,100185,006,m1,m1_rt,0.999756
672,407209,006,m1,m1_rt,0.999467
932,200164,006,m1,m1_rt,0.995130
3142,411477,006,m1,m1_rt,0.994132
3092,200166,006,m1,m1_rt,0.992711
332,100009,006,m1,m1_rt,0.987723
1022,100247,006,m1,m1_rt,0.986217
3613,100244,006,m1_rt,m1,0.982751
435,409381,003,m2_rt,m2,0.980498
3472,311047,006,m1,m1_rt,0.979639
